<a href="https://colab.research.google.com/github/matdjohnson-at-umass-dot-edu/cs646-hw3/blob/main/CS646_HW3_Fall_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[COMPSCI 646: Information Retrieval - Fall 2024 ](https://people.cs.umass.edu/~rahimi/teaching/fall2024/cs646.html)
#Assignment 3: Dense retrieval models (Total : 25 points)

**Description**
This is a coding assignment where you will write and execute code to run and evaluate a dense retriver model. Basic proficiency in Python and Pytorch is recommended.  

**Instructions**

* To start working on the assignment, you would first need to save the notebook to your local Google Drive. For this purpose, you can click on *Copy to Drive* button. You can alternatively click the *Share* button located at the top right corner and click on *Copy Link* under *Get Link* to get a link and copy this notebook to your Google Drive.  

*   You can download this notebook (file ipynb) and run it on your local machine, then upload the result file to Google Colab.

*   For questions with descriptive answers, please replace the text in the cell which states "Enter your answer here!" with your answer. If you are using mathematical notation in your answers, please define the variables.

*   For coding questions, you can add code where it says "enter code here" and execute the cell to print the output.



**Submission Details**

* Due date: Nov. 14, 5pm ET
* Before create the final PDF submission file, you need to provide the link to your Google Colab notebook. To get a publicly accessible Google Colab link, click the Share button in the top right corner, copy the link, and paste it here:


Link: https://colab.research.google.com/drive/1l16bfXgy2kEhOLEH6OJIelriFEy0jQlJ?usp=sharing

* To create the final PDF submission file, make sure to save all cell results you ran and generate a PDF using *File->Print->Save as PDF*. Make sure that the generated PDF contains all the codes and printed outputs before submission (You may need to change the print option to the Landscape orientation or adjust the scale of the PDF file, or you can use IPYNB to PDF converter). You are responsible for uploading the correct PDF with all the information required for grading.
* Upload the PDF file to [Gradescope](https://www.gradescope.com/courses/).


**Academic Honesty**

Please follow the guidelines under the *Collaboration and Help* section of the slides about course policy.

**Install library**

In [ ]:
!pip install transformers
!pip install faiss-cpu
!pip install datasets
!pip install pytrec_eval
!pip install --upgrade accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Use the code below to evaluate search results**






In [ ]:
import pytrec_eval

def get_scores(qrels, run):
  evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'P.10,100', 'ndcg_cut.10,100', 'recall.10,100'})
  result = evaluator.evaluate(run)
  metrics = ['P', 'ndcg_cut', 'recall']
  cutoffs = [10, 100]
  scores = {f'{metric}_{cutoff}': 0 for metric in metrics for cutoff in cutoffs}
  for key in result:
    for metric in metrics:
      for cutoff in cutoffs:
        scores[f'{metric}_{cutoff}'] += result[key][f'{metric}_{cutoff}']
  run_length = len(run)
  for score in scores:
    scores[score] /= run_length
  return scores

# 1: Vector similarity search using FAISS












We will start with a pre-trained dense retrieval model and use it to index and search over a dataset. For the retriever, we will use [Contriever](https://arxiv.org/pdf/2112.09118) with its implementation available [here](https://github.com/facebookresearch/contriever).
We will use the [FiQA dataset](https://huggingface.co/datasets/BeIR/fiqa) in the [BEIR Benchmark](https://arxiv.org/abs/2104.08663).








In [ ]:
import os
from collections import defaultdict
from typing import List, Dict
import numpy as np
import torch
import faiss
from transformers import AutoModel, AutoTokenizer

**Loading the dense retrieval model and tokenizer**

In [ ]:
model_name = "facebook/contriever-msmarco"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Loading the FiQA dataset**

We need to load ```corpus```, ```queries```, and ```qrels``` of the FiQA dataset, where:

*   ```corpus_ds```: HF dataset that have three colums `_id` (unique document ID), ```title```, and ```text``` (document content).
*   ```queries```: HF dataset that have three colums `_id` (unique query ID), ```title```, and ```text``` (query content).
*   ```qrels```: HF dataset that have three colums `query-id`, ```corpus-id```, and ```score``` (a relevant score between query and corpus).


In [ ]:
from datasets import load_dataset

queries_ds = load_dataset('BeIR/fiqa', 'queries')['queries']
qrels_ds = load_dataset('BeIR/fiqa-qrels')['test']
corpus_ds = load_dataset('BeIR/fiqa', 'corpus')['corpus']
# corpus_ds = load_dataset('BeIR/fiqa', 'corpus', split='corpus[:100]')

In [ ]:
#optional block to check data
print(queries_ds)
print(queries_ds[0])
print(corpus_ds)
print(corpus_ds[0])
print(qrels_ds)
print(qrels_ds[0])

Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 648
})
{'_id': '4641', 'title': '', 'text': 'Where should I park my rainy-day / emergency fund?'}
Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 57638
})
{'_id': '3', 'title': '', 'text': "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything."}
Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 1706
})
{'query-id': 8, 'corpus-id': 566392, 'score': 1}


The code below converts `qrels_ds` to `qrels`, which is a dictionary that maps each query ID (qid) to a dictionary of relevant documents (using their corpus IDs) along with their relevance scores.

We also need to filter `queries_ds` to include only the queries from the FiQA test set.

In [ ]:
qrels = {}
for row in qrels_ds:
    qid = str(row['query-id'])
    cid = str(row['corpus-id'])
    if qid not in qrels:
        qrels[qid] = {cid: row['score']}
    else:
        qrels[qid][cid] = row['score']

test_ids = list(qrels.keys())
queries_ds = queries_ds.filter(lambda x: x['_id'] in test_ids)

Filter:   0%|          | 0/648 [00:00<?, ? examples/s]

**BM25 Performance: the baseline for lexical matching**

We provide the BM25 retrieval results for the FiQA test set [here](https://drive.google.com/file/d/1NzidPMlViprYjUDDWPexpPXjXGiR5yx3/view?usp=sharing).

Run the code below to compare performance of the BM25 model with that of the dense retrieval model, which you will evaluate later.

In [ ]:
import json

bm25_fiqa_run_file = '/content/drive/MyDrive/CS646-Homework/HW3/bm25_fiqa_run.json'

with open(bm25_fiqa_run_file) as f:
  bm25_fiqa_run = json.load(f)

get_scores(qrels, bm25_fiqa_run)

{'P_10': 0.06342592592592625,
 'P_100': 0.012623456790123343,
 'ndcg_cut_10': 0.23606680758493984,
 'ndcg_cut_100': 0.2994274819068536,
 'recall_10': 0.29506844321659154,
 'recall_100': 0.5394860497869755}

**FAISS Index**

In this assignment, we will use the [FAISS library](https://github.com/facebookresearch/faiss) for efficient vector search.

We use **dot product** as vector similarity function.



The first step is to index vector representations of all passages in the collection.

**Question 1.1. Encoding Queries and Documents** (10 points)

First, we will use the  ```model``` loaded above to get the embeddings for all documents and test queries in the FiQA dataset. You will need to implement the function ```encode_texts``` to generate embeddings for a HF dataset that contain data (we will only consider the `text` field). The outputs are:

1.   `embeddings` which is an array of embeddings (```numpy``` array format) with shape `(M,D)`, where `M` is the number of rows in the input HF dataset, `D` is the embedding size, and each row represents the embedding for a single row.

2.   `ids` is the list of text IDs (from the `_id` field of the HF dataset), each corresponding to the representation of its respective text field in `embeddings`.


In [Contriever](https://arxiv.org/pdf/2112.09118), the same encoder is used to embed both queries and documents. "The representation for a query (resp. document) is obtained by averaging the hidden representations of the last layer."


We suggest encoding the text in batches to efficiently utilize the GPU for model inference. We also recommend encoding the texts in order of document length, as this can significantly improve encoding speed.

In [ ]:
import os
from collections import defaultdict
from typing import List, Dict
import numpy as np
import torch
import faiss
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import time
import sys


def encode_texts(model, tokenizer, text_ds, batch_size, info_log=False):
    predictions = None
    text_ids = list()
    upper_bound_addition_for_trailing_batch = 0
    if info_log:
      print("Entries to encode:")
      for i in range(0, len(text_ds)):
        print(text_ds[i])
    if len(text_ds) % batch_size != 0:
      upper_bound_addition_for_trailing_batch = 1
    batch_count = (len(text_ds) // batch_size) + upper_bound_addition_for_trailing_batch
    for i in range(0, batch_count):
        if i % 10 == 0:
          timestamp = time.strftime("%Y-%m-%dT%H:%M:%S", time.localtime())
          memory_usage = sys.getsizeof(predictions) / 1024 / 1024
          print(f"{timestamp} Processing batch {i+1} of {batch_count} - memory usage {memory_usage}Mb")
        batch = text_ds.select(range(i*batch_size, min(len(text_ds), (i+1)*batch_size)))
        tokenized_text = tokenizer(
            batch['text'],
            return_tensors="pt",
            padding='max_length',
            truncation=True,
            max_length=512
        )
        model_output = model(**tokenized_text)

        if info_log:
          print("Predictions")
          print(model_output.last_hidden_state.shape)
          print(model_output.last_hidden_state)
        batch_predictions = model_output.last_hidden_state.detach().cpu()
        batch_predictions_average = torch.mean(batch_predictions, dim=1, keepdim=False)
        if info_log:
          print("Predictions sum")
          print(batch_predictions_average.shape)
          print(batch_predictions_average)
        if predictions is None:
            predictions = batch_predictions_average.numpy()
        else:
            predictions = np.append(
                predictions,
                batch_predictions_average.numpy(),
                axis=0
            )
        if info_log:
          print("Predictions appended")
          print(predictions.shape)
          print(predictions)
        text_ids.extend(batch['_id'])
        del batch
        del tokenized_text
        del batch_predictions
    return predictions, text_ids


In [ ]:
#optional block to test
sample_text = corpus_ds.select(range(100))
embs, _ = encode_texts(model, tokenizer, sample_text, batch_size=20, info_log=True)
print('\nOutput embedding sizes:', embs.shape, embs[0].shape)
print('\nIDs length: ', len(_))

Entries to encode:
{'_id': '3', 'title': '', 'text': "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything."}
{'_id': '31', 'title': '', 'text': "So nothing preventing false ratings besides additional scrutiny from the market/investors, but there are some newer controls in place to prevent institutions from using them. Under the DFA banks can no longer solely rely on credit ratings as due diligence to buy a financial instrument, so that's a plus. The intent being that if financial institutions do their own leg work then *maybe* they'll figure out that a certain CDO is garbage or not.  E

Run the code below to get the embeddings for passages and test queries.

In [ ]:
passage_embeddings, passage_keys = encode_texts(model, tokenizer, corpus_ds, batch_size=32)
query_embeddings, query_keys = encode_texts(model, tokenizer, queries_ds, batch_size=32)

2024-11-14T16:13:19 Processing batch 1 of 1802 - memory usage 1.52587890625e-05Mb
2024-11-14T16:15:40 Processing batch 11 of 1802 - memory usage 0.9376220703125Mb
2024-11-14T16:18:05 Processing batch 21 of 1802 - memory usage 1.8751220703125Mb
2024-11-14T16:20:30 Processing batch 31 of 1802 - memory usage 2.8126220703125Mb
2024-11-14T16:22:52 Processing batch 41 of 1802 - memory usage 3.7501220703125Mb
2024-11-14T16:25:14 Processing batch 51 of 1802 - memory usage 4.6876220703125Mb
2024-11-14T16:27:42 Processing batch 61 of 1802 - memory usage 5.6251220703125Mb
2024-11-14T16:30:06 Processing batch 71 of 1802 - memory usage 6.5626220703125Mb
2024-11-14T16:32:34 Processing batch 81 of 1802 - memory usage 7.5001220703125Mb
2024-11-14T16:35:00 Processing batch 91 of 1802 - memory usage 8.4376220703125Mb
2024-11-14T16:37:27 Processing batch 101 of 1802 - memory usage 9.3751220703125Mb
2024-11-14T16:39:56 Processing batch 111 of 1802 - memory usage 10.3126220703125Mb
2024-11-14T16:42:22 Proc

In [ ]:
print('\nPassage embedding sizes:', passage_embeddings.shape, passage_embeddings[0].shape)
print('\nPassage keys length: ', len(passage_keys))
print('\nPassage embedding sizes:', query_embeddings.shape, query_embeddings[0].shape)
print('\nPassage keys length: ', len(query_keys))


Passage embedding sizes: (57638, 768) (768,)

Passage keys length:  57638

Passage embedding sizes: (648, 768) (768,)

Passage keys length:  648


**Question 1.2: Exact search (5 points)**

You will need to build a flat index of the corpus that is suitable for dot-product similarity search using FAISS. The "flat" (or brute-force) index stores all vectors directly in memory and compares each passage vector to the query vector during search. You will need to implement the `build_exact_search_index` below to get the flat index for `passage_embeddings`.

You also need to measure the memory usage for the built index.

In [ ]:
import sys

def build_exact_search_index(passage_embeddings):
    index_file = '/content/drive/MyDrive/CS646-Homework/HW3/exact_index.index'
    index = None
    index_size = None
    if os.path.exists(index_file):
        index = faiss.read_index(index_file)
        index_size = os.path.getsize(index_file)
    else:
        index = faiss.IndexFlatIP(passage_embeddings.shape[1])
        index.add(passage_embeddings)
        faiss.write_index(index, index_file)
        index_size = os.path.getsize(index_file)
    return index, index_size / 1024 / 1024


In [ ]:
exact_index, exact_memory_usage = build_exact_search_index(passage_embeddings)
print(f"Index entries: {exact_index.ntotal}")
print(f"Memory usage: {exact_memory_usage:.2f} MB")

Index entries: 57638
Memory usage: 168.86 MB


**Question 1.3: Retrieve results (5 points)**

Using the query embeddings and FAISS index of the passage collection, we can retrieve the top-$n$ ($n=100$) passages for each query.



 You can learn about these methods [here](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes).

 You will need to implement the `get_retrieval_run` function that retrieves the similar vectors (top 100) in the index for each query in `query_embeddings`.
You need to convert the results from the Faiss search (`distances` and `indices`) into the same format as the gold `qrels` for calculating retrieval scores.
You also need to measure the time taken for searching.


In [ ]:
def get_retrieval_run(index, query_embeddings, top_n=100):
    start_time = time.time()
    distances, indicies = index.search(query_embeddings, top_n)
    stop_time = time.time()
    search_time = stop_time - start_time
    qrels_run = {}
    for i in range(0, len(query_embeddings)):
        qrels_run[query_keys[i]] = {}
        for j in range(0, top_n):
            min_dist = np.min(distances[i])
            max_dist = np.max(distances[i])
            dist_normalized = (distances[i][j] - min_dist) / (max_dist - min_dist)
            assert dist_normalized >= 0 and dist_normalized <= 1
            qrels_run[query_keys[i]][passage_keys[indicies[i][j]]] = 1 if dist_normalized >= 0.75 else 0
    return qrels_run, search_time


Run the code below and report results.

In [ ]:
exact_run, exact_search_time = get_retrieval_run(exact_index, query_embeddings)
print(f"Memory usage for IndexFlatIP: {exact_memory_usage:.2f} MB")
print(f"Searching time: {exact_search_time} seconds")
print(get_scores(qrels, exact_run))

Memory usage for IndexFlatIP: 168.86 MB
Searching time: 0.8278472423553467 seconds
qrels: {'8': {'566392': 1, '65404': 1}, '15': {'325273': 1}, '18': {'88124': 1}, '26': {'285255': 1, '350819': 1}, '34': {'599545': 1}, '42': {'272709': 1, '327263': 1, '331981': 1}, '56': {'572690': 1}, '68': {'19183': 1}, '89': {'413229': 1, '590102': 1, '268026': 1, '248624': 1, '508754': 1, '64556': 1}, '90': {'31793': 1}, '94': {'245447': 1}, '98': {'575929': 1, '527522': 1}, '104': {'575869': 1, '523158': 1}, '106': {'76695': 1}, '109': {'73427': 1}, '475': {'366761': 1}, '503': {'367641': 1}, '504': {'500755': 1, '344203': 1, '498751': 1}, '515': {'372909': 1}, '529': {'510701': 1}, '547': {'6349': 1, '278629': 1}, '549': {'214024': 1}, '559': {'246459': 1}, '570': {'363591': 1}, '585': {'140226': 1, '552375': 1}, '588': {'570546': 1, '203710': 1}, '594': {'377322': 1, '534059': 1}, '603': {'456440': 1}, '604': {'451443': 1, '231947': 1, '261622': 1}, '620': {'331332': 1, '417301': 1, '189303': 1,

**Question 1.4:  Approximate nearest-neighbor search (5 points)**

Flat indexes provide exact results (by exhaustive search over the index)  but are not efficient. In this part, we use approximate nearest-neighbor search to improve search efficiency.

You will need to build an IVF index (`IndexIVFFlat`) in this part, and measure the memory usage of the index. This index has two parameters that can be adjusted: ```nproble``` and  ```nlist```. Please explain your choice of values for these two parameters in the text cell below. Note: tuning of these parameters is not needed for this assignment.

In [ ]:
def build_approximate_search_index(passage_embeddings):
    exact_index = faiss.IndexFlatIP(passage_embeddings.shape[1])
    index = faiss.IndexIVFFlat(
        exact_index,
        passage_embeddings.shape[1],
        int(np.ceil(np.sqrt(passage_embeddings.shape[1])))
    )
    index.train(passage_embeddings)
    index.add(passage_embeddings)
    temp_file = 'approx_temp.index'
    faiss.write_index(index, temp_file)
    return index, os.path.getsize(temp_file) / 1024 / 1024


Parameter values:

nproble: 768

nlist: ceil(sqrt(768)) = 28

The values were chosen to minimize computation time, given the observation that the number of Voronoi cells was required to be greater than or equal to the dimensionality of the embeddings (a dimensionality of 768), and that the number of cell visits could be lower than the total number of cells.


Run the code below and report results.


In [ ]:
approx_index, approx_memory_usage = build_approximate_search_index(passage_embeddings)
approx_run, approx_search_time = get_retrieval_run(approx_index, query_embeddings)
average_length = 0.0
for entry in qrels.keys():
  average_length += len(qrels[entry])
average_length /= len(qrels)
print(f"Memory usage for IndexIVFFlat: {approx_memory_usage:.2f} MB")
print(f"Searching time: {approx_search_time} seconds")
print(get_scores(qrels, approx_run))
print(f"Average length of qrels: {average_length}")

Memory usage for IndexIVFFlat: 169.38 MB
Searching time: 2.351818323135376 seconds
qrels: {'8': {'566392': 1, '65404': 1}, '15': {'325273': 1}, '18': {'88124': 1}, '26': {'285255': 1, '350819': 1}, '34': {'599545': 1}, '42': {'272709': 1, '327263': 1, '331981': 1}, '56': {'572690': 1}, '68': {'19183': 1}, '89': {'413229': 1, '590102': 1, '268026': 1, '248624': 1, '508754': 1, '64556': 1}, '90': {'31793': 1}, '94': {'245447': 1}, '98': {'575929': 1, '527522': 1}, '104': {'575869': 1, '523158': 1}, '106': {'76695': 1}, '109': {'73427': 1}, '475': {'366761': 1}, '503': {'367641': 1}, '504': {'500755': 1, '344203': 1, '498751': 1}, '515': {'372909': 1}, '529': {'510701': 1}, '547': {'6349': 1, '278629': 1}, '549': {'214024': 1}, '559': {'246459': 1}, '570': {'363591': 1}, '585': {'140226': 1, '552375': 1}, '588': {'570546': 1, '203710': 1}, '594': {'377322': 1, '534059': 1}, '603': {'456440': 1}, '604': {'451443': 1, '231947': 1, '261622': 1}, '620': {'331332': 1, '417301': 1, '189303': 1,

You need to compare the running time, memory usage, and quality (in terms of retrieval scores) of the two search methods mentioned above.

Exact index:
*   Search run time: 0.953299 seconds
*   Memory usage:    168.86 MB
*   Performance measures:
```
P_10:         0.000154
P_100':       0.000046
ndcg_cut_10:  0.000233
ndcg_cut_100: 0.000412
recall_10:    0.000257
recall_100:   0.000823
```


Approximate index:
*   Search run time: 2.622514 seconds
*   Memory usage:    169.38 MB
*   Performance measures:
```
P_10:         0.001697
P_100:        0.001543
ndcg_cut_10:  0.005750
ndcg_cut_100: 0.018903
recall_10:    0.010956
recall_100:   0.069708
```


Given the relatively low number of annotated relevant documents per query, use of recall is appropriate as this captures the low exepcted number of relevant documents per search result set.

The recall@10 of the approximate index is roughly 20 times that of the exact search index. The recall@100 of the approximate index is roughly 80 times that of the exact search index.

The approximate index appears to outperform the exact search index. This could be a result of the exact search index failing to visit Voronoi cells in the appropriate order. Another possible contributing factor is the semantic search functionality provided by the approximate search index, which increases the effective cross section of an evaluated query and relevant documents.

The cost of using approximate search is the time used to perform the computation. The approximate search is almost 3 times as costly in terms of computation time.